In [ ]:
!pip install vit-keras

In [ ]:
import keras
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
import vit_keras as vit

In [ ]:
IMAGES_DIR = ""
IMAGE_SIZE = (644, 644)
CLASSES = 93

In [ ]:
def gen_dataset(path, mask_subdir, batch_size, lab, input_shape):
    x = keras.utils.image_dataset_from_directory(os.path.join(path, "images"),
                                                 batch_size=1,
                                                 image_size=input_shape[:2],
                                                 crop_to_aspect_ratio=True,
                                                 labels=None).map(lambda x0: x0 / 255)#.map(lambda x1: tf.expand_dims(x1, 0) if len(x1.shape) == 3 else x1)
    y = keras.utils.image_dataset_from_directory(os.path.join(path, mask_subdir),
                                                 batch_size=1,
                                                 image_size=input_shape[:2],
                                                 crop_to_aspect_ratio=True,
                                                 labels=None,
                                                 color_mode='grayscale').map(lambda y: to_categorical(y, num_classes=CLASSES))
    print("Dataset Sizes:", len(x), len(y))
    datagen = tf.data.Dataset.zip((x, y))
    datagen = datagen.map(lambda x,y: (tf.squeeze(x, axis=0), tf.squeeze(y, axis=0))).batch(batch_size=batch_size, drop_remainder=True).prefetch(tf.data.AUTOTUNE)
    for s in datagen.take(1).as_numpy_iterator():
        print("X", s[0].shape, tf.reduce_max(s[0]).numpy())
        print("Y", s[1].shape, tf.reduce_max(s[1]).numpy())
    return datagen

In [ ]:
model = vit.vit_l32(
    image_size=IMAGE_SIZE[0],
    activation='sigmoid',
    pretrained=True,
    include_top=True,
    pretrained_top=False,
    classes=200
)